In [1]:
import generate_dataset
import encoder
import sys
sys.path.append("../")
from load_nii_open3d import cartesian_product, load_nii_voxels_segthy
from voxelgrid_slices import compute_slice_mesh_trimesh, slice_corners, slice_gridpoints
import torch
import trimesh
import numpy as np
import plotly.express as px
import matplotlib.cm
# import pyrender
import pandas as pd
from tqdm.notebook import tqdm
from datetime import datetime
import scipy
import scipy.io as sio
import pysdf

sys.path.append("../../Statistical Thyroid Model/Functional_maps_approach")
from eigenshapes import eigenshapes

#from slice_matching import (sample_on_plane_near_surface, create_slice_patches_dataset, create_sdf_patches_dataset, find_slice_mesh_matches,
#    procrustes_prediction, patch_cubes_mesh, show_mesh_with_slices, transform_slice_params, kde_estimate, restrict_mesh_z, hom_coords, PatchesDataset)
from slice_matching import show_mesh_with_slices
from slice_matching_meanshape import run_iterative_slice_matching_experiment_meanshape, run_soft_assignment_slice_matching_experiment_meanshape


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
############  Base dir for original data:
basedir = '/home/dilab/dilab_data/Thyroid Registration/Statistical Thyroid Model/Functional_maps_approach/data/out/with_only_train'
vals, vecs, mean, verts  = eigenshapes(basedir)
device = "cuda:1"

### get trasnslation for SSM samples
ty= np.zeros((0,3))
vert_t = []

for i in range(12):
    mat = scipy.io.loadmat(basedir + '/out_0_'+str(2*i)+'.mat')
    ty = np.vstack([ty, mat['translation_y']])
    vert_t.append(verts[:,:,i] + mat['translation_y'])   ### translated mesh vertices (5000, 3, 28)
vert_t = np.asarray(vert_t)
tavg = np.average(ty, axis = 0)


mean_meshvert = mean + tavg

# meanpcd = trimesh.points.PointCloud(mean_meshvert, colors = [100,0,100])
# meanpcd.scene().show()


['out_0_0.mat', 'out_0_2.mat', 'out_0_4.mat', 'out_0_6.mat', 'out_0_8.mat', 'out_0_10.mat', 'out_0_12.mat', 'out_0_14.mat', 'out_0_16.mat', 'out_0_18.mat', 'out_0_20.mat', 'out_0_22.mat']
rank of covariance matrix: 11


In [3]:
meanshape_mesh = generate_dataset.pcd2mesh(mean_meshvert)


shape = np.ceil(meanshape_mesh.vertices.max(axis=0) + np.array([64,64,64])).astype(int)
xp, yp, zp = [np.arange(0, shape[i]) for i in range(3)]
points = cartesian_product(xp, yp, zp)
f = pysdf.SDF(meanshape_mesh.vertices, meanshape_mesh.faces)
sdf_at_points = f(points)
meanshape_sdf = sdf_at_points.reshape(shape)

In [7]:
##### weighted soft margine triplet loss

#std = 0.05
model_path = 'saves/model_Nov28_22-06-30__lr=1e-05_batchSize=200_epochs=300_embeddingSize=128_usPatch=32x32x32_sdfPatch=32x32x32_normalized_reshuffle_pFurthest=0.2_loss=weightedSoftMarginTriplet(alpha=1.0)_embDist.obj'

model = torch.load(model_path, map_location=torch.device(device))
if isinstance(model, torch.nn.parallel.DataParallel):
    model = model.module

In [8]:
num_slice_patches = 100
num_sdf_patches = 100


### Original dataset
us_data_paths = [f"/home/dilab/dilab_data/data/{i}.nii" for i in range(0, 28, 2)]
mesh_paths = [f"/home/dilab/dilab_data/data/{i}.ply" for i in range(0, 28, 2)]

# ##### weighted soft margine triplet loss
run_iterative_slice_matching_experiment_meanshape(model, num_sdf_patches, num_slice_patches,
    model_path, us_data_paths, mesh_paths, meanshape_sdf, meanshape_mesh, device, smallest_k=50,z_restriction_width = 15, kde_bandwidth = 0.05,kde_k_largest=5,verbose=False)

251
sliceMatchingExperiment_Nov29_13-49-28_numSdfPatches=100_numSlicePatches=100_slicePatch=32x32x32_sdfPatch=32x32x32_numSlicesPerThyroid=50_smallestK=50_kdeKLargest=5_zRestrictionWidth=15_kdeBandwidth=0.05_model=Nov28_22-06-30_noPrevEffThresh_iter=2.csv


0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [9]:
# std = 0.5
model_path = 'saves/model_Nov28_23-04-13__lr=1e-05_batchSize=200_epochs=300_embeddingSize=128_usPatch=32x32x32_sdfPatch=32x32x32_normalized_reshuffle_pFurthest=0.2_loss=weightedSoftMarginTriplet(alpha=1.0)_embDist.obj'

model = torch.load(model_path, map_location=torch.device(device))
if isinstance(model, torch.nn.parallel.DataParallel):
    model = model.module


In [10]:
num_slice_patches = 100
num_sdf_patches = 100


### Original dataset
us_data_paths = [f"/home/dilab/dilab_data/data/{i}.nii" for i in range(0, 28, 2)]
mesh_paths = [f"/home/dilab/dilab_data/data/{i}.ply" for i in range(0, 28, 2)]

# ##### weighted soft margine triplet loss
run_iterative_slice_matching_experiment_meanshape(model, num_sdf_patches, num_slice_patches,
    model_path, us_data_paths, mesh_paths, meanshape_sdf, meanshape_mesh, device, smallest_k=50,z_restriction_width = 15, kde_bandwidth = 0.05,kde_k_largest=5,verbose=False)

251
sliceMatchingExperiment_Nov29_14-27-23_numSdfPatches=100_numSlicePatches=100_slicePatch=32x32x32_sdfPatch=32x32x32_numSlicesPerThyroid=50_smallestK=50_kdeKLargest=5_zRestrictionWidth=15_kdeBandwidth=0.05_model=Nov28_23-04-13_noPrevEffThresh_iter=2.csv


0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [11]:
# std = 1
model_path = 'saves/model_Nov29_09-08-06__lr=1e-05_batchSize=200_epochs=300_embeddingSize=128_usPatch=32x32x32_sdfPatch=32x32x32_normalized_reshuffle_pFurthest=0.2_loss=weightedSoftMarginTriplet(alpha=1.0)_embDist.obj'

model = torch.load(model_path, map_location=torch.device(device))
if isinstance(model, torch.nn.parallel.DataParallel):
    model = model.module

In [12]:
num_slice_patches = 100
num_sdf_patches = 100


### Original dataset
us_data_paths = [f"/home/dilab/dilab_data/data/{i}.nii" for i in range(0, 28, 2)]
mesh_paths = [f"/home/dilab/dilab_data/data/{i}.ply" for i in range(0, 28, 2)]

# ##### weighted soft margine triplet loss
run_iterative_slice_matching_experiment_meanshape(model, num_sdf_patches, num_slice_patches,
    model_path, us_data_paths, mesh_paths, meanshape_sdf, meanshape_mesh, device, smallest_k=50,z_restriction_width = 15, kde_bandwidth = 0.05,kde_k_largest=5,verbose=False)

251
sliceMatchingExperiment_Nov29_14-51-54_numSdfPatches=100_numSlicePatches=100_slicePatch=32x32x32_sdfPatch=32x32x32_numSlicesPerThyroid=50_smallestK=50_kdeKLargest=5_zRestrictionWidth=15_kdeBandwidth=0.05_model=Nov29_09-08-06_noPrevEffThresh_iter=2.csv


0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [ ]:
## Experiment 4      SSM with std0.5, changed num_patches to 400

model_path = 'saves/model_Nov29_15-28-15__lr=1e-05_batchSize=200_epochs=300_embeddingSize=128_usPatch=32x32x32_sdfPatch=32x32x32_normalized_reshuffle_pFurthest=0.2_loss=weightedSoftMarginTriplet(alpha=1.0)_embDist.obj'

model = torch.load(model_path, map_location=torch.device(device))
if isinstance(model, torch.nn.parallel.DataParallel):
    model = model.module

num_slice_patches = 100
num_sdf_patches = 100


### Original dataset
us_data_paths = [f"/home/dilab/dilab_data/data/{i}.nii" for i in range(0, 28, 2)]
mesh_paths = [f"/home/dilab/dilab_data/data/{i}.ply" for i in range(0, 28, 2)]

# ##### weighted soft margine triplet loss
run_iterative_slice_matching_experiment_meanshape(model, num_sdf_patches, num_slice_patches,
    model_path, us_data_paths, mesh_paths, meanshape_sdf, meanshape_mesh, device, smallest_k=50,z_restriction_width = 15, kde_bandwidth = 0.05,kde_k_largest=5,verbose=False)



251
sliceMatchingExperiment_Nov30_07-37-34_numSdfPatches=100_numSlicePatches=100_slicePatch=32x32x32_sdfPatch=32x32x32_numSlicesPerThyroid=50_smallestK=50_kdeKLargest=5_zRestrictionWidth=15_kdeBandwidth=0.05_model=Nov29_15-28-15_noPrevEffThresh_iter=2.csv


0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [8]:
## Experiment 5      SSM with std0.5, furthest to 0.8, check if it worked
model_path = 'saves/model_Nov30_01-06-54__lr=1e-05_batchSize=200_epochs=300_embeddingSize=128_usPatch=32x32x32_sdfPatch=32x32x32_normalized_reshuffle_pFurthest=0.8_loss=weightedSoftMarginTriplet(alpha=1.0)_embDist.obj'

model = torch.load(model_path, map_location=torch.device(device))
if isinstance(model, torch.nn.parallel.DataParallel):
    model = model.module

num_slice_patches = 100
num_sdf_patches = 100


### Original dataset
us_data_paths = [f"/home/dilab/dilab_data/data/{i}.nii" for i in range(0, 28, 2)]
mesh_paths = [f"/home/dilab/dilab_data/data/{i}.ply" for i in range(0, 28, 2)]

# ##### weighted soft margine triplet loss
run_iterative_slice_matching_experiment_meanshape(model, num_sdf_patches, num_slice_patches,
    model_path, us_data_paths, mesh_paths, meanshape_sdf, meanshape_mesh, device, smallest_k=50,z_restriction_width = 15, kde_bandwidth = 0.05,kde_k_largest=5,verbose=False)


251
sliceMatchingExperiment_Nov30_09-14-24_numSdfPatches=100_numSlicePatches=100_slicePatch=32x32x32_sdfPatch=32x32x32_numSlicesPerThyroid=50_smallestK=50_kdeKLargest=5_zRestrictionWidth=15_kdeBandwidth=0.05_model=Nov30_01-06-54_noPrevEffThresh_iter=2.csv


0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [6]:
## Experiment 6      SSM with std0.5, furthest to 0.5, check if it worked
model_path = 'saves/model_Nov30_09-45-27__lr=1e-05_batchSize=200_epochs=200_embeddingSize=128_usPatch=32x32x32_sdfPatch=32x32x32_normalized_reshuffle_pFurthest=0.5_loss=weightedSoftMarginTriplet(alpha=1.0)_embDist.obj'

model = torch.load(model_path, map_location=torch.device(device))
if isinstance(model, torch.nn.parallel.DataParallel):
    model = model.module

num_slice_patches = 100
num_sdf_patches = 100


### Original dataset
us_data_paths = [f"/home/dilab/dilab_data/data/{i}.nii" for i in range(0, 28, 2)]
mesh_paths = [f"/home/dilab/dilab_data/data/{i}.ply" for i in range(0, 28, 2)]

# ##### weighted soft margine triplet loss
run_iterative_slice_matching_experiment_meanshape(model, num_sdf_patches, num_slice_patches,
    model_path, us_data_paths, mesh_paths, meanshape_sdf, meanshape_mesh, device, smallest_k=50,z_restriction_width = 15, kde_bandwidth = 0.05,kde_k_largest=5,verbose=False)

251
sliceMatchingExperiment_Nov30_13-07-04_numSdfPatches=100_numSlicePatches=100_slicePatch=32x32x32_sdfPatch=32x32x32_numSlicesPerThyroid=50_smallestK=50_kdeKLargest=5_zRestrictionWidth=15_kdeBandwidth=0.05_model=Nov30_09-45-27_noPrevEffThresh_iter=2.csv


0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [7]:
## Experiment 7      SSM with std0.05, furthest to 0.5, check if it worked
model_path = 'saves/model_Nov30_13-40-24__lr=1e-05_batchSize=200_epochs=200_embeddingSize=128_usPatch=32x32x32_sdfPatch=32x32x32_normalized_reshuffle_pFurthest=0.5_loss=weightedSoftMarginTriplet(alpha=1.0)_embDist.obj'


model = torch.load(model_path, map_location=torch.device(device))
if isinstance(model, torch.nn.parallel.DataParallel):
    model = model.module

num_slice_patches = 100
num_sdf_patches = 100


### Original dataset
us_data_paths = [f"/home/dilab/dilab_data/data/{i}.nii" for i in range(0, 28, 2)]
mesh_paths = [f"/home/dilab/dilab_data/data/{i}.ply" for i in range(0, 28, 2)]

# ##### weighted soft margine triplet loss
run_iterative_slice_matching_experiment_meanshape(model, num_sdf_patches, num_slice_patches,
    model_path, us_data_paths, mesh_paths, meanshape_sdf, meanshape_mesh, device, smallest_k=50,z_restriction_width = 15, kde_bandwidth = 0.05,kde_k_largest=5,verbose=False)

251
sliceMatchingExperiment_Nov30_21-42-49_numSdfPatches=100_numSlicePatches=100_slicePatch=32x32x32_sdfPatch=32x32x32_numSlicesPerThyroid=50_smallestK=50_kdeKLargest=5_zRestrictionWidth=15_kdeBandwidth=0.05_model=Nov30_13-40-24_noPrevEffThresh_iter=2.csv


0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [8]:
## Experiment 8      SSM with std1, furthest to 0.5, check if it worked
model_path = 'saves/model_Nov30_17-14-48__lr=1e-05_batchSize=200_epochs=200_embeddingSize=128_usPatch=32x32x32_sdfPatch=32x32x32_normalized_reshuffle_pFurthest=0.5_loss=weightedSoftMarginTriplet(alpha=1.0)_embDist.obj'


model = torch.load(model_path, map_location=torch.device(device))
if isinstance(model, torch.nn.parallel.DataParallel):
    model = model.module

num_slice_patches = 100
num_sdf_patches = 100


### Original dataset
us_data_paths = [f"/home/dilab/dilab_data/data/{i}.nii" for i in range(0, 28, 2)]
mesh_paths = [f"/home/dilab/dilab_data/data/{i}.ply" for i in range(0, 28, 2)]

# ##### weighted soft margine triplet loss
run_iterative_slice_matching_experiment_meanshape(model, num_sdf_patches, num_slice_patches,
    model_path, us_data_paths, mesh_paths, meanshape_sdf, meanshape_mesh, device, smallest_k=50,z_restriction_width = 15, kde_bandwidth = 0.05,kde_k_largest=5,verbose=False)

251
sliceMatchingExperiment_Nov30_22-07-50_numSdfPatches=100_numSlicePatches=100_slicePatch=32x32x32_sdfPatch=32x32x32_numSlicesPerThyroid=50_smallestK=50_kdeKLargest=5_zRestrictionWidth=15_kdeBandwidth=0.05_model=Nov30_17-14-48_noPrevEffThresh_iter=2.csv


0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [4]:

## Experiment 9      meanshape meanshape
model_path = 'saves/model_Nov30_23-40-22__lr=1e-05_batchSize=200_epochs=200_embeddingSize=128_usPatch=32x32x32_sdfPatch=32x32x32_normalized_reshuffle_pFurthest=0.5_loss=weightedSoftMarginTriplet(alpha=1.0)_embDist.obj'


model = torch.load(model_path, map_location=torch.device(device))
if isinstance(model, torch.nn.parallel.DataParallel):
    model = model.module

num_slice_patches = 100
num_sdf_patches = 100


### Original dataset
us_data_paths = [f"/home/dilab/dilab_data/data/{i}.nii" for i in range(0, 28, 2)]
mesh_paths = [f"/home/dilab/dilab_data/data/{i}.ply" for i in range(0, 28, 2)]

# ##### weighted soft margine triplet loss
run_iterative_slice_matching_experiment_meanshape(model, num_sdf_patches, num_slice_patches,
    model_path, us_data_paths, mesh_paths, meanshape_sdf, meanshape_mesh, device, smallest_k=50,z_restriction_width = 15, kde_bandwidth = 0.05,kde_k_largest=5,verbose=False)




251
sliceMatchingExperiment_Dec01_12-43-49_numSdfPatches=100_numSlicePatches=100_slicePatch=32x32x32_sdfPatch=32x32x32_numSlicesPerThyroid=50_smallestK=50_kdeKLargest=5_zRestrictionWidth=15_kdeBandwidth=0.05_model=Nov30_23-40-22_noPrevEffThresh_iter=2.csv


0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [23]:
print("Thyroid extents:", np.min(meanshape_mesh.vertices, axis=0), "-", np.max(meanshape_mesh.vertices, axis=0))
slice_params = (np.array((200., 150, 210)), np.array((1,0,0)), np.array((0,1,0)), 200., 200.)
slice_corner_points = np.array(slice_corners(*slice_params), dtype=int)
# print("Slice corners:", slice_corner_points)

show_mesh_with_slices(meanshape_mesh, [slice_params])

Thyroid extents: [46.19930322 44.0048148  55.30329604] - [250.35952613 220.86225973 401.46698236]


In [7]:
# sdf_dir = f"/home/dilab/dilab_data/data/data_ssmsdf/oridataset/sdf_train"

# meanshape_pc=torch.load(sdf_dir+'/SSMverts')[6]

# meanshape_mesh = ssm_thyroid_dataset.pcd2mesh(meanshape_pc)

# shape = np.ceil(meanshape_mesh.vertices.max(axis=0) + np.array([64,64,64])).astype(int)
# xp, yp, zp = [np.arange(0, shape[i]) for i in range(3)]
# points = cartesian_product(xp, yp, zp)
# f = pysdf.SDF(meanshape_mesh.vertices, meanshape_mesh.faces)
# sdf_at_points = f(points)
# meanshape_sdf = sdf_at_points.reshape(shape)

In [8]:
meanshape_mesh.scene().show()

# Legacy stuff (ignore, unless needed for debugging)

#### for us_scan_path : get US scan data path ####  .nii.gz files

UStrain_data_path = "/home/dilab/dilab_data/data/subset_SegThy/left/US/train/"
USval_data_path = "/home/dilab/dilab_data/data/subset_SegThy/left/US/val/"


# ### for sdf_paths : get SDF data path ### tensors

sdf_train_path = "/home/dilab/dilab_data/data/data_ssmsdf/sdf_train/"
sdf_val_path = "/home/dilab/dilab_data/data/data_ssmsdf/sdf_val/"

## label paths
USlabel_path = "/home/dilab/dilab_data/data/subset_SegThy/left/US/labels/"

# ## for mean shape
# basedir='/home/dilab/dilab_data/Thyroid Registration/Statistical Thyroid Model/Functional_maps_approach/data/out/2022_10_17_10_21_46/mat'

basedir= '/home/dilab/dilab_data/Thyroid Registration/Statistical Thyroid Model/Functional_maps_approach/data/out/2022_10_31_10_49_21/mat'

device = "cuda:0"
USindex = 0
idx = 0
ultrasound = load_nii_voxels_segthy(UStrain_data_path + "/0"+str(USindex+1).zfill(2)+"_P1_1_left_US.nii.gz")

#sdf = torch.load(data_path + "sdf_data/data%d/sdf" % data_index)
sdf = torch.load(sdf_train_path + "SSMsample" + str(idx) + "sdf")

#mesh = trimesh.load_mesh(data_path + "/%d.ply" % data_index)
mesh = meanshape_mesh

num_slice_patches = 300
num_sdf_patches = 900
slice_patch_size = np.array([32,32,32])
sdf_patch_size = np.array([32,32,32])
slice_patches_dataset = create_slice_patches_dataset(mesh, ultrasound, slice_params, num_slice_patches, slice_patch_size, perturb_std_dev=2.0)
sdf_patches_dataset = create_sdf_patches_dataset(mesh, sdf, num_sdf_patches, sdf_patch_size)

threshold_config = { # exactly one of the two first parameters can be active at the same time
    # "dists_thresh": 0.11,
    "smallest_k": 200,
    "only_one_match_per_patch": True
}
slice_matches, sdf_matches, slice_match_coords, sdf_match_coords = find_slice_mesh_matches(
    model, slice_patches_dataset, sdf_patches_dataset, device, embedding_dimension=128, sdf_batch_size=64, **threshold_config)

def slice_match_once():
    transformation_matrix = procrustes_prediction(sdf_match_coords, slice_match_coords)
    predicted_slice_params = transform_slice_params(slice_params, transformation_matrix)

    patch_cubes = patch_cubes_mesh(slice_match_coords, sdf_match_coords, slice_patch_size, sdf_patch_size)
    show_mesh_with_slices(mesh, [slice_params, predicted_slice_params], additional_meshes=[patch_cubes])
# slice_match_once()

z_projection = np.array([[0],[0],[1]])
kde_estimate(sdf_match_coords, z_projection, show_plot=True, all_data=sdf_patches_dataset.patches_coords, k_largest=5, kde_bandwidth=0.05)

procrustes_predictions = iterative_slice_matching(mesh, ultrasound, sdf, slice_params, 300, 900,
                                                  slice_patch_size, sdf_patch_size, 20, 0.05, kde_k_largest=5, use_previous_effective_threshold=False, smallest_k=120, num_iterations=3)

def show_mesh_with_slices_pyrender(mesh, slices_params,
        color_sequence=((0, 1.0, 0), (1.0, 1.0, 0)) + matplotlib.cm.Set2.colors,
        camera_angles=(-0.2, 1.15, 0), camera_distance=750, camera_center=(0, 0, 0),
        additional_meshes=[]
    ):
    scene = pyrender.Scene()
    for m in ([mesh] + [compute_slice_mesh_trimesh(*slice_params, color=color)
            for slice_params, color in zip(slices_params, color_sequence)]
            + additional_meshes):
        scene.add(pyrender.Mesh.from_trimesh(m, smooth=False))
    pyrender.Viewer(scene, use_raymond_lighting=True)

# procrustes_transform_idx = 5
# best_procrustes_transform = procrustes_predictions[procrustes_transform_idx][0]
# print(procrustes_predictions[procrustes_transform_idx][1])
procrustes_predictions_sorted = sorted(procrustes_predictions, key=lambda p: p[1])
procrustes_transform_slices = [transform_slice_params(slice_params, transform) for transform, _ in procrustes_predictions_sorted]
procrustes_scores_normalized = np.array([score for _, score in procrustes_predictions_sorted])
procrustes_scores_normalized = (procrustes_scores_normalized - procrustes_scores_normalized.min()) / (
    procrustes_scores_normalized.max() - procrustes_scores_normalized.min())

# predicted_slice_params = transform_slice_params(slice_params, best_procrustes_transform)

#patch_cubes = patch_cubes_mesh(slice_match_coords, sdf_match_coords, slice_patch_size, sdf_patch_size)
# predicted_slice_colors = np.minimum(1, matplotlib.cm.RdYlGn_r(procrustes_scores_normalized) * 2)
# predicted_slice_colors = np.minimum(1, matplotlib.cm.RdYlGn_r(np.linspace(0.1, 1, len(procrustes_scores_normalized)) * 3))
predicted_slice_colors = np.linspace([1.,1.,0],[0.2,0.2,0.2], len(procrustes_predictions_sorted)) ** 1.2
color_sequence = [(0, 1.0, 0)] + list(map(tuple, predicted_slice_colors))
show_mesh_with_slices(mesh, [slice_params] + procrustes_transform_slices, additional_meshes=[], color_sequence=color_sequence)

def slices_mean_distance(slice_params1, slice_params2, resolution=100):
    slice_gridpoints1 = slice_gridpoints(*slice_params1, num_steps1=resolution, num_steps2=resolution)
    slice_gridpoints2 = slice_gridpoints(*slice_params2, num_steps1=resolution, num_steps2=resolution)
    return np.linalg.norm(slice_gridpoints1 - slice_gridpoints2, axis=0).mean()

slice_gridpoints1 = slice_gridpoints(*slice_params)

[slices_mean_distance(slice_params, matched_slice_params) for matched_slice_params in procrustes_transform_slices]

def run_iterative_slice_matching_experiment(num_sdf_patches, num_slice_patches, slice_patch_size = np.array([32,32,32]), sdf_patch_size = np.array([32,32,32]),
    num_slices_per_thyroid = 50, smallest_k = None, thresh=None, kde_k_largest = 5, z_restriction_width = 10, kde_bandwidth = 0.05, use_previous_effective_threshold=False, thyroid_range=list(range(28)),
                                           sdf_batch_size=75, iterations=2, only_one_match_per_patch=True):
    results_list = []
    torch.cuda.empty_cache()

    hyperparams = [
        ("numSdfPatches", num_sdf_patches),
        ("numSlicePatches", num_slice_patches),
        ("slicePatch", "x".join(map(str, slice_patch_size))),
        ("sdfPatch", "x".join(map(str, sdf_patch_size))),
        ("numSlicesPerThyroid", num_slices_per_thyroid),
        ("smallestK", smallest_k),
        ("thresh", thresh),
        ("kdeKLargest", kde_k_largest),
        ("zRestrictionWidth", z_restriction_width),
        ("kdeBandwidth", kde_bandwidth),
        ("model", model_path[model_path.rfind("/")+1+6:model_path.rfind("/")+1+20]),
        (None, "prevEffThresh" if use_previous_effective_threshold else "noPrevEffThresh"),
        ("iter", iterations),
    ]

    date_str = datetime.now().strftime("%b%d_%H-%M-%S")
    log_filename = "sliceMatchingExperiment_" + date_str + "_" + "_".join((key + "=" if key is not None else "") + str(value) for key, value in hyperparams if value is not None) + ".csv"
    print(len(log_filename))
    print(log_filename)

    for data_index in tqdm(thyroid_range):
        ultrasound = load_nii_voxels(data_path + "%d.nii" % data_index)
        sdf = torch.load(data_path + "sdf_data/data%d/sdf" % data_index)
        mesh = trimesh.load_mesh(data_path + "/%d.ply" % data_index)
        label = load_nii_voxels(data_path + "%d-labels.nii" % data_index)

        for current_z_index, current_z in tqdm(enumerate(np.linspace(mesh.vertices[:,2].min() + 20, mesh.vertices[:,2].max() - 20, num_slices_per_thyroid))):
            slice_params = (np.array((mesh.vertices[:,0].mean(), mesh.vertices[:,1].mean(), current_z)), np.array((1.,0,0)), np.array((0,1,0)), 200., 200.)
            procrustes_predictions = iterative_slice_matching(mesh, ultrasound, sdf, slice_params, num_slice_patches, num_sdf_patches,
                slice_patch_size, sdf_patch_size, z_restriction_width, kde_bandwidth, kde_k_largest, verbose=False, smallest_k=smallest_k, dists_thresh=thresh, sdf_batch_size=sdf_batch_size,
                num_iterations=iterations, use_previous_effective_threshold=use_previous_effective_threshold, only_one_match_per_patch=only_one_match_per_patch)
            procrustes_predictions = sorted(procrustes_predictions, key=lambda p: p[1]) # sort by loss
            procrustes_transform_slices = [transform_slice_params(slice_params, transform) for transform, _ in procrustes_predictions]
            # current_results.append([slices_mean_distance(slice_params, matched_slice_params) for matched_slice_params in procrustes_transform_slices])
            # print(data_index, "z =", current_z, ":", current_results[-1])
            for candidate_index in range(len(procrustes_predictions)):
                results_list.append([data_index, current_z, current_z_index, candidate_index, procrustes_transform_slices[candidate_index],
                    *procrustes_predictions[candidate_index], slices_mean_distance(slice_params, procrustes_transform_slices[candidate_index])])

        results_df = pd.DataFrame(results_list, columns=["thyroid_index", "z_value", "z_index", "candidate_index", "candidate_slice_params",
            "candidate_transform", "candidate_procrustes_loss", "candidate_slices_mean_distance"])
        results_df.to_csv(log_filename)

# run_iterative_slice_matching_experiment(num_sdf_patches=300, num_slice_patches=100, slice_patch_size = np.array([32,32,32]), sdf_patch_size = np.array([32,32,32]),
#    num_slices_per_thyroid = 50, smallest_k = 50, kde_k_largest = 5, z_restriction_width = 10, kde_bandwidth = 0.05, use_previous_effective_threshold=True)

run_iterative_slice_matching_experiment(num_sdf_patches=300, num_slice_patches=100, slice_patch_size = np.array([32,32,2]), sdf_patch_size = np.array([32,32,32]),
    num_slices_per_thyroid = 50, smallest_k=50, kde_k_largest = 5, z_restriction_width = 10, kde_bandwidth = 0.05, sdf_batch_size=10)

model_path = "saves/model_Jul17_23-22-14__lr=0.001_batchSize=100_epochs=60_embeddingSize=128_usPatch=32x32x8_sdfPatch=32x32x32_unnormalized_reshuffle_posPerturb=10.0_loss=geometricDistanceL2(alpha=5.0)_negSampleThreshold=40.0.obj"
model = None
torch.cuda.empty_cache()
model = torch.load(model_path, map_location=torch.device(device))

run_iterative_slice_matching_experiment(num_sdf_patches=300, num_slice_patches=100, slice_patch_size = np.array([32,32,8]), sdf_patch_size = np.array([32,32,32]),
    num_slices_per_thyroid = 50, smallest_k=50, kde_k_largest = 5, z_restriction_width = 10, kde_bandwidth = 0.05, sdf_batch_size=100)

model_path = "saves/model_Jul23_21-46-02__lr=0.001_batchSize=10_epochs=60_embeddingSize=128_usPatch=64x64x2_sdfPatch=64x64x32_unnormalized_reshuffle_posPerturb=10.0_loss=geometricDistanceL2(alpha=5.0)_negSampleThreshold=40.0.obj"
model = None
torch.cuda.empty_cache()
model = torch.load(model_path, map_location=torch.device(device))

run_iterative_slice_matching_experiment(num_sdf_patches=300, num_slice_patches=100, slice_patch_size = np.array([64,64,2]), sdf_patch_size = np.array([64,64,32]),
    num_slices_per_thyroid = 50, smallest_k=50, kde_k_largest = 5, z_restriction_width = 10, kde_bandwidth = 0.05, sdf_batch_size=10)

model_path = "saves/model_Jul21_09-42-41__lr=0.001_batchSize=10_epochs=60_embeddingSize=128_usPatch=128x128x2_sdfPatch=128x128x32_unnormalized_reshuffle_posPerturb=10.0_loss=geometricDistanceL2(alpha=5.0)_negSampleThreshold=40.0.obj"
model = None
torch.cuda.empty_cache()
model = torch.load(model_path, map_location=torch.device(device))

run_iterative_slice_matching_experiment(num_sdf_patches=300, num_slice_patches=100, slice_patch_size = np.array([128,128,2]), sdf_patch_size = np.array([128,128,32]),
    num_slices_per_thyroid = 50, smallest_k=50, kde_k_largest = 5, z_restriction_width = 10, kde_bandwidth = 0.05, sdf_batch_size=10)

# PART OF RUN: 50/100;300 128x128x(2/32) distance matching loss thyroid 16, 20

# Tensorboard: Jul21_09-42-41_narvis-compute_lr=0.001_batchSize=10_epochs=60_embeddingSize=128_usPatch=128x128x2_sdfPatch=128x128x32_unnormalized_reshuffle_posPerturb=10.0_loss=geometricDistanceL2(alpha=5.0)_negSampleThreshold=40.0
model_path = "saves/model_Jul21_09-42-41__lr=0.001_batchSize=10_epochs=60_embeddingSize=128_usPatch=128x128x2_sdfPatch=128x128x32_unnormalized_reshuffle_posPerturb=10.0_loss=geometricDistanceL2(alpha=5.0)_negSampleThreshold=40.0.obj"
model = None
torch.cuda.empty_cache()
model = torch.load(model_path, map_location=torch.device(device))

run_iterative_slice_matching_experiment(num_sdf_patches=300, num_slice_patches=100, slice_patch_size = np.array([128,128,2]), sdf_patch_size = np.array([128,128,32]),
    num_slices_per_thyroid = 50, smallest_k=50, kde_k_largest = 5, z_restriction_width = 10, kde_bandwidth = 0.05, sdf_batch_size=9,
                                       thyroid_range=list(range(17,20)))

model_path = "saves/model_Jul23_21-46-02__lr=0.001_batchSize=10_epochs=60_embeddingSize=128_usPatch=64x64x2_sdfPatch=64x64x32_unnormalized_reshuffle_posPerturb=10.0_loss=geometricDistanceL2(alpha=5.0)_negSampleThreshold=40.0.obj"
model = None
torch.cuda.empty_cache()
model = torch.load(model_path, map_location=torch.device(device))

run_iterative_slice_matching_experiment(num_sdf_patches=300, num_slice_patches=100, slice_patch_size = np.array([64,64,2]), sdf_patch_size = np.array([64,64,32]),
    num_slices_per_thyroid = 50, smallest_k=50, kde_k_largest = 5, z_restriction_width = 10, kde_bandwidth = 0.05, sdf_batch_size=10, thyroid_range=list(range(23,28)))


run_iterative_slice_matching_experiment(num_sdf_patches=3000, num_slice_patches=1000, slice_patch_size = np.array([32,32,32]), sdf_patch_size = np.array([32,32,32]),
    num_slices_per_thyroid = 50, smallest_k = 500, kde_k_largest = 5, z_restriction_width = 10, kde_bandwidth = 0.05, use_previous_effective_threshold=True, thyroid_range=range(21, 28))

model_path = "saves/model_Jul16_00-30-17__lr=0.001_batchSize=100_epochs=60_embeddingSize=128_usPatch=32x32x32_sdfPatch=32x32x32_unnormalized_reshuffle_posPerturb=10.0_loss=geometricDistanceL2(alpha=5.0)_negSampleThreshold=40.0.obj"
model = None
torch.cuda.empty_cache()
model = torch.load(model_path, map_location=torch.device(device))

run_iterative_slice_matching_experiment(num_sdf_patches=300, num_slice_patches=100, slice_patch_size = np.array([32,32,32]), sdf_patch_size = np.array([32,32,32]),
    num_slices_per_thyroid = 10, smallest_k = 50, kde_k_largest = 5, z_restriction_width = 10, kde_bandwidth = 0.05, use_previous_effective_threshold=False, iterations=2,
                                       only_one_match_per_patch=False)

model_path = "saves/model_Jul13_06-34-07__lr=0.001_batchSize=10_epochs=60_embeddingSize=128_usPatch=50x50x8_sdfPatch=50x50x32_normalized_reshuffle_posPerturb=10.0_loss=weightedSoftMarginTriplet(alpha=5.0)_negSampleThreshold=40.0.obj"
model = None
torch.cuda.empty_cache()
model = torch.load(model_path, map_location=torch.device(device))

run_iterative_slice_matching_experiment(num_sdf_patches=300, num_slice_patches=100, slice_patch_size = np.array([50,50,8]), sdf_patch_size = np.array([50,50,32]),
    num_slices_per_thyroid = 50, smallest_k = 50, kde_k_largest = 5, z_restriction_width = 10, kde_bandwidth = 0.05, use_previous_effective_threshold=True)

model_path = "saves/model_Jul13_20-47-25__lr=0.001_batchSize=10_epochs=60_embeddingSize=128_usPatch=64x64x8_sdfPatch=64x64x32_normalized_reshuffle_posPerturb=10.0_loss=weightedSoftMarginTriplet(alpha=5.0)_negSampleThreshold=40.0.obj"
model = None
torch.cuda.empty_cache()
model = torch.load(model_path, map_location=torch.device(device))

run_iterative_slice_matching_experiment(num_sdf_patches=300, num_slice_patches=100, slice_patch_size = np.array([64,64,8]), sdf_patch_size = np.array([64,64,32]),
    num_slices_per_thyroid = 50, smallest_k = 50, kde_k_largest = 5, z_restriction_width = 10, kde_bandwidth = 0.05, use_previous_effective_threshold=True, sdf_batch_size=12)

model_path = "saves/model_Jul18_11-09-45__lr=0.001_batchSize=25_epochs=60_embeddingSize=128_usPatch=64x64x8_sdfPatch=64x64x32_unnormalized_reshuffle_posPerturb=10.0_loss=geometricDistanceL2(alpha=5.0)_negSampleThreshold=40.0.obj"
model = None
torch.cuda.empty_cache()
model = torch.load(model_path, map_location=torch.device(device))

run_iterative_slice_matching_experiment(num_sdf_patches=300, num_slice_patches=100, slice_patch_size = np.array([64,64,8]), sdf_patch_size = np.array([64,64,32]),
    num_slices_per_thyroid = 50, thresh=16.45, kde_k_largest = 5, z_restriction_width = 10, kde_bandwidth = 0.05, use_previous_effective_threshold=True, sdf_batch_size=12, iterations=3)

model_path = "saves/model_Jul18_11-09-45__lr=0.001_batchSize=25_epochs=60_embeddingSize=128_usPatch=64x64x8_sdfPatch=64x64x32_unnormalized_reshuffle_posPerturb=10.0_loss=geometricDistanceL2(alpha=5.0)_negSampleThreshold=40.0.obj"
model = None
torch.cuda.empty_cache()
model = torch.load(model_path, map_location=torch.device(device))

slice_params = (np.array((mesh.vertices[:,0].mean(), mesh.vertices[:,1].mean(), current_z)), np.array((1.,0,0)), np.array((0,1,0)), 200., 200.)
procrustes_predictions = iterative_slice_matching(mesh, ultrasound, sdf, slice_params, 300, 900,
    slice_patch_size, sdf_patch_size, 20, 0.05, kde_k_largest=5, use_previous_effective_threshold=False, smallest_k=120, num_iterations=3)